In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Reports System

## Default Usage

OpenMDAO has a reports system which will generate reports when you run your model. To see a list of the
reports that are run by default, along with a brief description of each, use the command:

```
openmdao list_reports -d
```

Running the same command without the `-d` will list all reports that are available.

Here is a table with more info about these default commands.

| Report Name      | Description                                                  | When Run                       | Doc link                                                           |
|------------------|--------------------------------------------------------------|--------------------------------|--------------------------------------------------------------------|
| `inputs`         | Interactive table of model inputs                            | after `Problem.final_setup`    |                                                                     |
| `n2`             | Interactive, graphical view of the model                     | after `Problem.final_setup`    | [N2 diagram](../model_visualization/n2_basics/n2_basics.ipynb)     |
| `optimizer`      | Summary of results of an optimization run                    | after `Drive._post_run`        | [optimization report](optimization_report.ipynb)  
| `scaling`        | Information on design variables, objectives, and constraints | after `Driver._compute_totals` | [scaling report](../model_visualization/view_scaling_report.ipynb) |
| `total_coloring` | Metadata associated with the creation of the coloring        | after `Driver._get_coloring`   |                                                                    |


Note that the reports are only run the first time the method is called on a given instance. For example, the scaling report is only run after the first call to `Driver._compute_totals`.

These reports will, by default, be put into a subfolder of `reports` named after the `Problem` name. 
For example, if your model has a single Problem and its default name, `problem1`, was not changed by the user, the reports will be placed in a directory named `reports/problem1`.

The user has control over the reports system by setting some environment variables which are described below.

## Controlling which reports are generated

There are two ways to control which reports are generated on which models.

(features:reports:report_system:controlling_reporting_using_an_environment_variable)=
### Controlling reporting using an environment variable
To turn off OpenMDAO's default report generation, the user should set the `OPENMDAO_REPORTS`
environment variable to any one of these five case-insensitive values: '0', 'false', 'no', 'off', or 'none'.

The user can also have more fine-grained control over what reports are generated by setting this environment variable to a string that lists the reports to run. For example

1. OPENMDAO_REPORTS=n2

2. OPENMDAO_REPORTS=n2,scaling

3. OPENMDAO_REPORTS=none

If `OPENMDAO_REPORTS` is unset, the default reports will be generated. 

The user can also explicitly turn on all available reports by setting `OPENMDAO_REPORTS` to 'all'.

### Controlling reporting using the Problem reports argument 
The user can also control what reports are generated using the `reports` argument to `Problem`.

If `reports` is the default of `_UNDEFINED` or `True`, the `OPENMDAO_REPORTS` environment variable determines reporting.

If the value for `reports` is `None` or `False`, it will disable all reports on that `Problem`.

Otherwise, `reports` may be a list of names or a single comma-delimited string of the names of the reports to run for that Problem.

Overall, `OPENMDAO_REPORTS` represents the global default list of reports, and individual Problems can either use that default list or override it by specifying their own list explicitly.

## Setting the directory for the reports

If the user wishes to have the reports directory placed into a different directory other than the default current working directory, the user can set the environment variable, `OPENMDAO_REPORTS_DIR` to the name of that directory. The directory does not have to exist beforehand. It will be created by the reporting system. As an example, if `OPENMDAO_REPORTS_DIR` is set to "my_reports", then, if the `Problem` name is `problem1`, then the reports will be written to the "my_reports/problem1" directory.

## Support for subproblems

The reporting system supports subproblems. For example, if a model has two Problems and they have the names `problem1` and `problem2`, the reporting system will put the reports into the subfolders, `problem1` and `problem2` under the top level reports directory, which defaults to `./reports` or is specified by the value of `OPENMDAO_REPORTS_DIR`.

## Adding user defined reports

The user can register a user defined report using the `register_report` function.

```{eval-rst}
    .. autofunction:: openmdao.utils.reports_system.register_report
        :noindex:
```

Here is a script to show how this is done. Note that the function that is called to generate the actual report must have as its first argument the OpenMDAO object instance that owns the method where the report will be run. Currently, the only option is a `Problem` or `Driver` instance. The full file path of the report will be determined based on `OPENMDAO_REPORTS_DIR` and the name of the enclosing `Problem` instance.  The directory where the report will be written can
be obtained by calling the `get_reports_dir` method on the instance passed into the reporting function.

Note that `register_report` only adds a report type to the global reports registry.  In order to activate the report so that it actually generates output, it must appear in either `OPENMDAO_REPORTS` or must be passed in the `Problem` reports arg as shown below.

After this script is run, there will be a file named `user_report.txt` in the directory `problem1`.


In [ ]:
import os
os.environ['OPENMDAO_REPORTS'] = '1'  # Need to run reports in this notebook.

In [ ]:
import pathlib
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

user_report_filename = 'user_report.txt'

def user_defined_report(prob):
    path = pathlib.Path(prob.get_reports_dir()).joinpath(user_report_filename)
    with open(path, "w") as f:
        f.write(f"Do some reporting on the Problem, {prob._name}\n")

# Run the report before the call to Problem.setup
om.register_report("user_report", user_defined_report, "This report is user defined",
                   'Problem', 'setup', 'pre')

prob = om.Problem(reports='user_report')
model = prob.model

model.add_subsystem('p1', om.IndepVarComp('x', 0.0), promotes=['x'])
model.add_subsystem('p2', om.IndepVarComp('y', 0.0), promotes=['y'])
model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])

model.add_design_var('x', lower=0.0, upper=1.0)
model.add_design_var('y', lower=0.0, upper=1.0)
model.add_objective('f_xy')

prob.driver = om.ScipyOptimizeDriver()

prob.setup()
prob.run_driver()
prob.cleanup()

In [ ]:
try:
    om.unregister_report('user_report')
except KeyError:
    pass

import pathlib
path = pathlib.Path(prob.get_reports_dir()).joinpath(user_report_filename)
assert path.is_file(),"Could not find problem report"

There are some advanced cases where it may be necessary to register more then one hook for a report. This happens currently with the 'n2' report, which may run after `setup()` if there are errors before or during setup, or after `final_setup()` if there are no errors.  In cases where multiple hooks are necessary, multiple calls to `register_report_hook` should be used instead of a single call to `register_report`.  The signature for `register_report_hook` is:

```{eval-rst}
    .. autofunction:: openmdao.utils.reports_system.register_report_hook
        :noindex:
```


## Creating a report plugin

What if you've developed an OpenMDAO-related python package and would like anyone using that package
to have access to a new report type that you've defined within that package?  You can do this by
creating an `openmdao_report` plugin using the steps outlined below.  This example will use the
existing `summary` report plugin as an example.

1) Create the function that generates your report. For example:

    ```python
    def _summary_report(prob):
        path = str(pathlib.Path(prob.get_reports_dir()).joinpath('summary.html'))
        s = StringIO()
        config_summary(prob, s)
        with open(path, 'w') as f:
            f.write(text2html(s.getvalue()))
    ```

    Note that your report function must take as an argument an instance of the type that contains the
    method where your report is registered to run.  For example, in the `register_report` call below,
    it refers to the `final_setup` method of `Problem`, so our instance in this case must be an 
    instance of `Problem`.  Also, note that the function calls `prob.get_reports_dir()` to get the
    location of the reports directory where the report shouild be written.  In addition, the
    string generated from calling `config_summary` is plain text, so it is wrapped in html using the
    `text2html` function to make it easily viewable in a browser, and therefore accessible using the
    `openmdao view_reports` command.

2) Create a function that will register your new report type.  For example:

    ```python
    def _summary_report_register():
        register_report('summary', _summary_report, 'Model summary', 'Problem', 'final_setup', 'post')
    ```

    The call above will result in a report type called `summary` that triggers at the end of
    `Problem.final_setup`.


3) Create an entry point in your package's `setup.py` file.  In this example, our report
registry function is found in the `openmdao.devtools.debug.py` file, so we'll update the 
`entry_point` arg in the call to `setup` in our `setup.py` file as follows:

    ```python
        entry_points = {
            
            # other entry point groups here...
            
            'openmdao_report': [
                
                # other report entry points here...
                
                'summary=openmdao.devtools.debug:_summary_report_register',
            ],
        }
    ```

    As shown above, report entry points are part of the `openmdao_report` entry point group.


## Viewing existing reports

As mentioned earlier, the `openmdao view_reports` command provides an easy way to view all of the
reports that have been generated, either for an entire script or just for a specific Problem 
instance.  To view only reports for a specific problem, use

```
openmdao view_reports probname
```

where `probname` is the name of the problem of interest.

## Listing available reports

As mentioned before, the user can list available reports using the `openmdao list_reports` command.

It can also be done programmatically, using the  `list_reports` function.

```{eval-rst}
    .. autofunction:: openmdao.utils.reports_system.list_reports
        :noindex:
```

In [ ]:
om.list_reports()

In [ ]:
os.environ['OPENMDAO_REPORTS'] = '0'